## Goal

Our goal is that given a user's playlist history and a target playlist, we want to generate a list of song recommendations that should maintain the trend of `energy` (we only consider the direction of the trend) and the generated playlist should be similar to the target as close as possible, but there may be some other limitations, such as the number of commonly listened songs to include. 

## Methods
There are a total of four proposed methods:

- **Pure method**: <u> Submit queries to Spotify Get Recommendations API and get a list of recommendations for each song in the target playlist, and then filter these recommendations according to the trend of energy.</u> This means that if the second song has a smaller energy than the first one in the target playlist, the second recommended song should also have a smaller energy than the first recommended song. 

- **Meet-middle method**: Since the Pure method starts from the first song, the difference between the target plalylist and the list of recommended songs will enlarge as the number of songs increases. To alleviate this issue, I proposed the **Meet-middle method**. In this method, <u>I first create several buckets based on time</u>. For example, for a target playlist with a total duration 63 mins, I can create 10 mins, 15 mins, 20 mins, 10 mins, and 8 mins buckets. <u>Within each bucket, we generate two lists of recommendations, one of which starts from the first song and the other one starts from the last song. For bucket that has more than one songs, we then try to find a way to connect these two lists of recommendations to generate the final list of recommendations for the whole target playlist.</u> To do this, within each bucket, we start from the middle song, we check whether the energy trend is satisfied. If so, we can connect the two lists at the middle place. If not, we will move to the next song that is close to the middle one, and redo the checking. And finally, we can output the final list of recommended songs.
<img src="meetmiddle.png" alt="drawing" style="width:550px;"/>

- **Recommend first, then place common songs**: <u>Use a **modified version of Pure method** to generate a list of recommendations. Then replace some recommendations by common songs. </u> To do this, we compare the "difference" between each recommendation and each common song and finally replace some recommendations with common songs based on similarity.

- **Place common songs first, then recommend**:<u> Compare the "difference" between each target song and each common song and determine the places for some common songs. Then use **Meet-middle method** to generate recommendations between common songs. 

### Helper functions

1. Get **a user**'s playlist IDs (and maybe added_time): `get_plst_ID`
2. Get **a Playlist**'s Items, including song_id and duration: `get_ID`
3. Get information about **a song**. If you only want to know "name", "popularity" and "duration_ms" about a song, use function `get_song_name`. To obtain more comprehensive information about a song, use function `get_song`. (Note: `get_song_name` is only used for displaying song's frequency in the playlist history now.)
4. Get **a user**'s common songs from playlist history: `common`
5. Get Recommendations for **a song** using Spotify API: `recommend`
6. Find places for common songs: `place_common`
7. Create an empty playlist in the user's Spotify account: `create_plst`



### Pure method
Use `get_recommendation_for_a_playlist` and add additional parameters `common_prop = 0` or just skip this parameter.

### Meet-middle method
Use `meet_middle`.

### Recommend first, then place common songs
Use `RecommendPlace`

### Place common songs first, then recommend
Use `PlaceRecommend`

## ======================  Code ============================================

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime, timezone
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

#### p1. Get a user's playlists (and added_time)

In [2]:
def get_plst_ID(user_id, token, market, added_time = False):
    tidy = lambda s: s[17:]
    
    base_url = "https://api.spotify.com/v1/users/"
    query = f'{base_url}{user_id}/playlists'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    plst_id = [tidy(x['uri'])for x in json_response['items']]
    if not added_time:
        return plst_id
    else:
        base_url = "https://api.spotify.com/v1/playlists/"
        add = []
        for idx in plst_id:
            query = f'{base_url}{idx}/tracks?market={market}'
            response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
            json_response = response.json()
            add.append(json_response['items'][0]['added_at'])
        return(plst_id, add)

#### p2. Get a Playlist's Items: song_id and duration

In [3]:
def get_ID(user_id, token, playlist_id, market):
    tidy = lambda s: s[14:]
    base_url = "https://api.spotify.com/v1/playlists/"
    query = f'{base_url}{playlist_id}/tracks?market={market}'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['track']['uri'])for x in json_response['items']], [json_response['items'][i]['track']['duration_ms'] for i in range(len(json_response['items']))]

#### p3. Get information about a song  (Dec 16 updated)

In [4]:
def get_song(user_id, token, song_id, market):
    
    ## get audio features of the song/track
    song_url = "https://api.spotify.com/v1/audio-features/"
    query = f'{song_url}{song_id}'
    success = False
    while not success:
        try:
            response = requests.get(query, 
                           headers={"Content-Type":"application/json", 
                                    "Authorization":f"Bearer {token}"})
            json_response = response.json()
            df_response = pd.json_normalize(json_response)
            success = True
        except Exception as e:
            print(e)
    
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    success = False
    while not success:
        try:
            response = requests.get(query, 
                           headers={"Accept": "application/json",
                                    "Content-Type":"application/json", 
                                    "Authorization":f"Bearer {token}"})
            json_response = response.json()
            album_id = json_response['album']['id'] if 'album' in json_response.keys() else None
            artist_id = json_response['artists'][0]['id'] if 'artists' in json_response.keys() else None
            success = True
        except Exception as e:
            print(e)
    
    ## get genre of the album
    base_url = "https://api.spotify.com/v1/albums/"
    query = f'{base_url}{album_id}?market={market}'
    success = False
    while not success:
        try:
            response = requests.get(query, 
                           headers={"Accept": "application/json",
                                    "Content-Type":"application/json", 
                                    "Authorization":f"Bearer {token}"})
            json_response = response.json()
            genre = json_response['genres'] if 'genres' in json_response.keys() else None
            success = True
        except Exception as e:
            print(e)
                
    df_response['artist_id'] = artist_id if artist_id is not None and len(artist_id) >0 else None
    df_response['albuma_id'] = album_id if album_id is not None and len(album_id) >0 else None
    df_response['genre'] = genre if genre is not None and len(genre) >0 else None
    try:
        return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')
    except Exception:
        print(song_id)

In [5]:
def get_song_name(user_id, token, song_id, market):
    
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    df_response = pd.json_normalize(json_response)
    return df_response[['name', 'popularity', 'duration_ms', 'id']].set_index('id')

#### p4. Get a user's common songs from playlist history (Dec 5 updated)

In [6]:
def convert_weight(time_list, half_decay = 1):
    ''' Calculate weights.'''
    ## I use exponential decay now.
    ## the weight decays to 50% every `half_decay` days
    today = datetime.now(timezone.utc)
    log_weights = np.array([- (today - parse(s)).days - (today - parse(s)).seconds / 86400 for s in time_list])
    log_weights -= np.max(log_weights)
    return np.exp(log_weights / half_decay * np.log(2))

def get_song_freq(user_id, token, market, start_plst, end_plst, max_num = 100, use_time = True, decay_rate = 15):
    plst_ids, add = get_plst_ID(user_id, token, market, added_time = True)
    plst_ids, add = plst_ids[start_plst:(end_plst+1)], add[start_plst:(end_plst+1)]
    if use_time:
        weights = convert_weight(add, decay_rate)
    else:
        weights = [np.exp(-0.5*x) for x in range(len(add))]
    ids = [get_ID(user_id, token, playlist, market)[0] for playlist in plst_ids]
    song_list = [(s, w) for ls, w in list(zip(ids, weights)) for s in ls]
    counts = {}
    for s, w in song_list:
        counts[s] = counts.get(s, 0) + w
    df = pd.DataFrame(sorted(counts.items(), key = lambda x: -x[1]), columns = ['id', 'weighted_freq'])
    df = df.set_index('id').iloc[:max_num]
    dff = pd.concat([get_song_name(user_id, token, idx, market = 'US') for idx in df.index])
    return(pd.merge(df, dff, how = 'left', on = 'id')).reset_index().set_index('name')

def count_songs(user_id, token, market):
    plst_ids= get_plst_ID(user_id, token, market)
    ids = [get_ID(user_id, token, playlist, market)[0] for playlist in plst_ids]
    return(len(ids))

def common(top, user_id, token, market, **kwargs):
    num_songs = count_songs(user_id, token, market)
    top = round(num_songs * top)
    kwargs.pop('max_num', None)        
    freq_df = get_song_freq(user_id, token, market, max_num = top, **kwargs)
    return(list(freq_df.iloc[:top].reset_index()['id']))

#### p5. Get Recommendations for a song using Spotify API

In [7]:
def check(par, par_range):
    ''' Check if values are in the feasible range. '''
    return(np.clip(par, par_range[0], par_range[1]))

def check_whole(kwargs):
    ''' Run the above check function to all optional parameters. '''
    pars = ['valence', 'speechiness', 'acousticness', 'liveness', 'danceability']
    ranges = {'valence': [0,1], 'speechiness': [0,1], 'acousticness': [0,1], 'liveness': [0,1], 'danceability': [0,1]}
    if len(kwargs) == 0:
        return(kwargs)
    else:
        for item in pars:
            if np.array([re.match(f"(.*?)({item})", x) is not None for x in kwargs.keys()]).any(): 
                item_min = 'min_' + item
                item_max = 'max_' + item
                item_target = 'target_' + item
                par_range = ranges.get(item)
                kwargs[item_target] = check(kwargs.get(item_target), par_range)
                kwargs[item_min] = check(kwargs.get(item_min), par_range)
                kwargs[item_max] = check(kwargs.get(item_max), par_range)
        return(kwargs)
    
def recommend(song, limit, target_duration_ms, require_percents, optional_percents, **kwargs):
    seed_tracks = song.index[0]
    
    time_mt = require_percents.get('time')
    energy_percent = require_percents.get('energy')
    instrumentalness_percent = require_percents.get('instrumentalness')
    tempo_percent = require_percents.get('tempo')
    
    min_duration_ms = int(max(target_duration_ms - time_mt * 1000, 0))
    max_duration_ms = int(target_duration_ms + time_mt * 1000)
    
    ## required percents
    target_energy = song['energy'][0]
    min_energy = check(target_energy * (1 - energy_percent), [0,1])
    max_energy = check(target_energy * (1 + energy_percent), [0,1])
    target_instrumentalness = song['instrumentalness'][0]
    min_instrumentalness = check(target_instrumentalness * (1 - instrumentalness_percent), [0,1])
    #max_instrumentalness = check(target_instrumentalness * (1 + instrumentalness_percent), [0,1])
    target_tempo = song['tempo'][0]
    min_tempo = check(target_tempo * (1 - tempo_percent), [0, np.inf])
    max_tempo = check(target_tempo * (1 + tempo_percent), [0, np.inf])

    ## default parameters
    target_key = song['key'][0]
    target_danceability=song['danceability'][0]
    target_mode=song['mode'][0]
         
    ## optional percents
    for item in optional_percents: 
        item_min = 'min_' + item
        item_max = 'max_' + item
        item_target = 'target_' + item
        target = song[item][0]
        kwargs[item_target] = target
        if item not in ['mode', 'key', 'popularity', 'time_signature']:
            kwargs[item_min] = target * (1 - optional_percents.get(item))
            kwargs[item_max] = target * (1 + optional_percents.get(item))
    kwargs.pop('optional_percents', None)   ## remove `optional_percents` from kwargs
    kwargs = check_whole(kwargs)
    
    endpoint_url = "https://api.spotify.com/v1/recommendations?"
    query = f'{endpoint_url}limit={limit}&seed_tracks={seed_tracks}'
    query += f'&target_duration_ms={target_duration_ms}&min_duration_ms={min_duration_ms}&max_duration_ms={max_duration_ms}'
    query += f'&target_energy={target_energy}&min_energy={min_energy}&max_energy={max_energy}'
    query += f'&target_instrumentalness={target_instrumentalness}&min_instrumentalness={min_instrumentalness}'
    query += f'&target_tempo={target_tempo}&min_tempo={min_tempo}&max_tempo={max_tempo}'
    query += f'&target_key={target_key}'
    query += f'&target_danceability={target_danceability}'
    query += f'&target_mode={target_mode}'
    
    if song['genre'][0] is not None:
        seed_genres = song['genre'][0]
        query += f'&seed_genres={seed_genres}'
    if song['artist_id'][0] is not None:
        seed_artist = song['artist_id'][0]
        query += f'&seed_artist={seed_artist}'

    if len(kwargs.keys()) > 0:
        query += '&'
        lst = [str(x[0])+'='+str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all limits from kwargs
    uris = [] 
    #print(query)
    
    success = False
    while not success:
        try:
            response = requests.get(query, 
                                    headers={"Content-Type":"application/json", 
                                             "Authorization":f"Bearer {token}"})
            json_response = response.json()
            #print('Recommended Songs:')
            uris = []
            for i,j in enumerate(json_response['tracks']):
                uris.append(j['uri'])
                #print(f"{i+1}) \"{j['name']}\" by {j['artists'][0]['name']}")
            success = True
        except Exception as e:
            print(e)

    return uris

#### p6. Find places for common songs

In [8]:
def place_common(user_id, token, market, common_songs, num, playlist_id=None, songs_id=None, pars_weight = None):
    if playlist_id is not None:
        ids, _ = get_ID(user_id, token, playlist_id, market)
    elif songs_id is not None:
        ids = songs_id
    else:
        NameError('Please input either playlist_id or songs_id.')
        
    pars = ['energy', 'duration_ms', 'instrumentalness', 'danceability', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'liveness', 'valence', 'tempo', 'time_signature']
    if pars_weight is None:
        cols = ['energy', 'duration_ms', 'instrumentalness']
        weight = [0.5, 0.4, 0.1] 
    else:
        cols = list(filter(lambda x: x in pars, pars_weight.keys()))
        weight = [pars_weight[key] for key in cols]
        weight = list(np.array(weight) / np.sum(weight))
    
    ids = [x[14:] for x in songs_id]
    common_songs = [x[14:] for x in common_songs]
    dff = pd.concat([get_song(user_id, token, idx, market) for idx in common_songs])._get_numeric_data()[cols]
    
    candidate = {key: (None, np.inf) for key in dff.index}
    for i in range(len(ids)):
        if dff.shape[0] < 1:
            break
        idx = ids[i]
        info = get_song(user_id, token, idx, market)[cols]
        diff = ((dff.sub(info._get_numeric_data().values[0].tolist(), axis='columns').applymap(np.square))*weight).sum(axis = 1).sort_values()
        for j in range(diff.shape[0]):
            if diff[j] < candidate[diff.index[j]][1]:
                candidate[diff.index[j]] = (i, diff[j])
    
    sorted_candidate = sorted(candidate.items(),key=lambda x: x[1][1])
    output = {key: values[0] for (key, values) in sorted_candidate}
    return(['spotify:track:' + x for x in list(output.keys())[:num]], list(output.values())[:num])

#### p7. Create a playlist

In [9]:
# create a new playlist to store those recommendations
def create_plst(user_id, token, uris, name, description, public = False):
    endpoint_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    request_body = json.dumps({
        "name": name,
        "description": description,
        "public": public })
    # create an empty new playlist
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    url = response.json()['external_urls']['spotify']
    if response.status_code == 201:
        print('Playlist {} is successfully created!'.format(name))
        
    # fill the new playlist with the recommendations
    playlist_id = response.json()['id']
    endpoint_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    request_body = json.dumps({ "uris" : uris })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    if response.status_code == 201:
        print('Playlist {} is successfully filled with recommendations!'.format(name))
        print(f'Your playlist is ready at {url}')

## Main functions

### Generate a similar playlist

In [10]:
def get_recommendation_for_a_playlist(user_id, token, limit, name='',description='', playlist_id=None, songs_id=None, create_pl=False, public=False, quiet=False, **kwargs):
    if not quiet:
        print("playlist", playlist_id)
    
    market = kwargs.get("market", 'US')
    kwargs["market"] = market
    
    common_top = kwargs.get("common_top", 0.25) ## default: songs that have a frequency within the top 25% are regarded as common songs 
    kwargs.pop("common_top", None)
    start_plst = kwargs.get("start_plst", 0)
    end_plst = kwargs.get("end_plst", 20)
    kwargs.pop("start_plst", None)
    kwargs.pop("end_plst", None)
    common_songs = ['spotify:track:'+ x for x in common(common_top, user_id, token, market = market, start_plst = start_plst, end_plst = end_plst)]
    #print(common_songs) 
    
    percent_args = kwargs.get("percent_args", {})
    requires = ['time', 'energy', 'instrumentalness', 'tempo']
    require_percents = {}
    optional_percents = {}
    for item in requires:  ## for required percents, extract them and store in `require_percents`
        require_percents[item] = percent_args.get(item, 0.1)
    for item in list(set(percent_args) - set(requires)): ## for optional percents, store them in `optional_percents`
        optional_percents[item] = percent_args.get(item)
    kwargs.pop('percent_args', None)        
    if not quiet:
        print("args:", kwargs, require_percents, optional_percents)
    
    def filter_energy(r, pre_energy, market, trend = None):
    
        if trend is None or pre_energy == 0:
            return r[0]

        energys_diff = [get_song(user_id, token, r[i][14:], market)['energy'][0] - pre_energy for i in range(len(r))]
        if trend == '+':
            if all(np.array(energys_diff) <= 0):
                return None
            idx = np.argmax( np.array(energys_diff) > 0)
        else:
            if all(np.array(energys_diff) >= 0):
                return None
            idx = np.argmax( np.array(energys_diff) < 0)
        return(r[idx])

    def _get_one_recommendation(idx, limit, target_duration_ms, require_percents, optional_percents, **kwargs):
        df_response = get_song(user_id, token, idx, market)
        r = recommend(df_response, limit, target_duration_ms, require_percents, optional_percents, **kwargs)
        if not quiet:
            print(f"For the {j+1}-th song, no recommendations, enlarged the range about duration.")
        new_time_percent = require_percents.get('time')
        new_energy_percent = require_percents.get('energy')
        new_require_percents = require_percents.copy()
        i = 0
        while len(r) == 0 and i < 10:
            i += 1
            new_time_percent += 10
            new_energy_percent *= 2
            new_require_percents['time'] = new_time_percent
            #new_require_percents['energy'] = new_energy_percent
            r = recommend(df_response, limit, target_duration_ms, new_require_percents, optional_percents, **kwargs)
        if i == 10:
            ## right now I keep the original one, we can improve it later
            r = ['spotify:track:' + df_response.index[0]]
            ## another way is to remove all optional limitations
            # r = recommend(df_response, limit, target_duration_ms, require_percents, {}, **kwargs)
        return r
        
    if playlist_id is not None:
        ## if playlist_id is specified, songs_id will not be used.
        ids, durations = get_ID(user_id, token, playlist_id, market)
    elif songs_id is not None:
        ids = songs_id
        if 'spotify:track:' in ids[0]:
            ids = [x[14:] for x in ids]
        durations = [get_song(user_id, token, ids[j], market)['duration_ms'][0] for j in range(len(ids))]
    else:
        NameError('Please input either playlist_id or songs_id.')
    
    common_num = round(kwargs.get("common_prop", 0) * len(ids)) ## default: don't use common song to substitute recommendations
    kwargs.pop("common_prop", None)
    if not quiet:
        print('common songs to be included: ', common_num)
    
    uris = []
    duration = 0
    diff = 0
    pre_energy = 0
    ene = 0
    trend = None
    current_common_num = 0
    candidate = []
    for j in range(len(ids)):
        energy = get_song(user_id, token, ids[j], market)['energy'][0] 
        trend = '+' if energy > pre_energy else '-'
        _limit = limit
        _r = None
        i = 0
        while _r is None and i < 10:
            ## if no songs satisfy requirements, we get more recommendations from API and then filter
            r = _get_one_recommendation(ids[j], _limit, durations[j]-diff, require_percents, optional_percents, **kwargs)    
            ######### if a common song exists in the recommendations, we will adopt it, rather than filtering based on energy trend
            if common_num > current_common_num and len(set.intersection(set(common_songs), set(r))) > 0:
                _r = [i for i in common_songs if i in r][0]
                current_common_num += 1
                if not quiet:
                    print('common song', _r[14:], 'is used.')
                break
            _r = filter_energy(r, ene, market, trend) ## add limitations to filter the recommendations and keep the energy trend
            if len(r) < _limit: ## no matter how to increase the limitation on the number of recommendations, result will no change
                break
            _limit = _limit + 10
            i = i+1
        candidate.append(r)
        if i == 10 or _r is None:
            _r = r[0] ## if still no songs satisfy requirements, use the most similar song from Spotify Recommendation API
        if not quiet:
            print('candidate: ', [x[14:] for x in r])
            print('recommendation: ', _r[14:])
        uris.append(_r)
        dur = get_song(user_id, token, _r[14:], market)['duration_ms'][0] ## duration of the recommended song
        duration += dur
        diff = duration - sum(durations[:j+1])
        ene = get_song(user_id, token, _r[14:], market)['energy'][0] ## energy of the recommended song
        pre_energy = energy  ## energy of the j-th song in the playlist

    if create_pl:
        create_plst(user_id, token, [x for x in uris if x is not None], name, description, public)
    
    if common_num == 0:
        return(uris, candidate)
    
    else:
        used_common = list(filter(lambda x: x in uris, common_songs))
        return(uris, candidate, common_songs, used_common, common_num, current_common_num)

### Meet middle (Dec 16 updated)

In [11]:
def meet_middle(user_id, token, limit, bucket_lengths, name='', description='', playlist_id = None, songs_id = None, create_pl=False, public=False, **kwargs):
    market = kwargs.get("market", 'US')
    kwargs["market"] = market
    
    if playlist_id is not None: ## if playlist_id is specified, songs_id will not be used.
        ids, durations = get_ID(user_id, token, playlist_id, market)
    elif songs_id is not None:
        ids = songs_id
        if 'spotify:track:' in ids[0]:
            ids = [x[14:] for x in ids]
        durations = [get_song(user_id, token, ids[j], market)['duration_ms'][0] for j in range(len(ids))]
    else:
        NameError('Please input either playlist_id or songs_id.')
    
    
    def create_bucket(user_id, token, ids, bucket_lengths = None):
        durations = [get_song(user_id, token, song_id, market)['duration_ms'][0] for song_id in ids]
        if bucket_lengths is None: ## default: the whole playlist is a bucket
            bucket_lens = [sum(durations)] 
        elif (type(bucket_lengths) == int): ## buckets with the same length
            print(np.sum(durations) // bucket_lengths + 1)
            bucket_lens = [bucket_lengths * 60 * 1000 for i in range(int(np.sum(durations) / 60000 // bucket_lengths + 1))]
        elif (len(bucket_lengths) == 1): ## buckets with the same length
            bucket_lens = [bucket_lengths[0] * 60 * 1000 for i in range(int(np.sum(durations) / 60000// bucket_lengths[0] + 1))]
        else:
            bucket_lens = [x * 60 * 1000 for x in bucket_lengths]
        bucket = 0
        end_songs = [None] * len(bucket_lens)
        cur_length = 0
        for e, l in enumerate(durations):
            if np.abs(cur_length + l - bucket_lens[bucket]) < np.abs(cur_length - bucket_lens[bucket]):
                end_songs[bucket] = e
                cur_length += l
            else:
                bucket += 1
                if bucket == len(end_songs):
                    break
                end_songs[bucket] = e
                cur_length += l - bucket_lens[bucket - 1]
        start = 0
        buckets = []
        for i in end_songs:
            buckets.append(ids[start:(i+1)])
            start = i+1
        return(buckets)
    
    
    def meet_middle_for_bucket(user_id, token, limit, songs_id, **kwargs):
        f_pl, _ = get_recommendation_for_a_playlist(user_id, token, limit=limit, name='', description='', songs_id=songs_id, **kwargs)
        b_pl, _ = get_recommendation_for_a_playlist(user_id, token, limit=limit, name='', description='', songs_id=songs_id[::-1], **kwargs)
        b_pl = b_pl[::-1]
        if len(f_pl) == 1 or f_pl == b_pl:
            return([x[14:] for x in f_pl])
        else:
            f_pl = [x[14:] for x in f_pl]
            b_pl = [x[14:] for x in b_pl]
            middle = len(f_pl) // 2
            energys = [get_song(user_id, token, x, market)['energy'][0]  for x in songs_id]
            trends = [1 if energys[i+1] > energys[i] else -1 for i in range(len(energys)-1)]
            f_energy = get_song(user_id, token, f_pl[middle-1], market)['energy'][0] 
            b_energy = get_song(user_id, token, b_pl[middle], market)['energy'][0]
            trend = trends[middle-1]
            if f_pl[middle] == b_pl[middle] or trend * (b_energy - f_energy) > 0:
                return(f_pl[:middle] + b_pl[middle:])
            else:
                offset = 1
                finished = None
                while trend * (b_energy - f_energy) <= 0 and middle+1+offset < len(b_pl)-1:
                    f_energy = get_song(user_id, token, f_pl[middle-1+offset], market)['energy'][0] 
                    b_energy = get_song(user_id, token, b_pl[middle+1+offset], market)['energy'][0]
                    trend = trends[middle-1+offset]
                    if trend * (b_energy - f_energy) > 0:
                        return(f_pl[:(middle+1)] + b_pl[(middle+1):])
                    else:
                        offset += 1
            if finished is None:
                offset = -1
                while trend * (b_energy - f_energy) <= 0 and middle-1+offset > 0:
                    f_energy = get_song(user_id, token, f_pl[middle-1+offset], market)['energy'][0] 
                    b_energy = get_song(user_id, token, b_pl[middle+1+offset], market)['energy'][0]
                    trend = trends[middle-1+offset]
                    if trend * (b_energy - f_energy) > 0:
                        return(f_pl[:(middle+1)] + b_pl[(middle+1):])
                    else:
                        offset += -1
            if finished is None:
                ### if after searching from the middle to the end and the beginning, no points between 
                ### these two playlists can be combined, we just use the original one that goes from the beginning
                return (f_pl)
    
    if (type(bucket_lengths) == int):
        bucket_lengths = [bucket_lengths]
    if int(max(np.ceil(sum(durations) / 60000- sum(bucket_lengths)), 0)) > 0:
        bucket_lengths = bucket_lengths + [int(np.ceil(sum(durations) / 60000- sum(bucket_lengths)))]
    # create buckets
    buckets = create_bucket(user_id, token, ids = ids, bucket_lengths = bucket_lengths)
    recommendations_pl = []
    for song_id in buckets:
        pl = meet_middle_for_bucket(user_id, token, limit, song_id, quiet = True, **kwargs)
        recommendations_pl.append(pl)
        
    ## flattern
    recommendations_pl = [num for elem in recommendations_pl for num in elem]
    ## add something so that Spotify API know what it is 
    recommendations_pl = ['spotify:track:'+ str(x) for x in recommendations_pl]
    
    if create_pl:
        create_plst(user_id, token, uris=recommendations_pl, name=name, description=description, public=public)
    
    return recommendations_pl

### Recommend first, then place common songs

In [12]:
def RecommendPlace(user_id, token, limit, name='',description='', playlist_id=None, songs_id=None, create_pl=False, public=False, **kwargs):
    
    pars_weight = kwargs.get("pars_weight", {'energy': 0.4, 'duration_ms': 0.4, 'danceability': 0.2})
    kwargs.pop('pars_weight', None)
    create_pl = kwargs.get("create_pl", create_pl)
    kwargs.pop('create_pl', None)
    
    song_recom, candidate, common_songs, used_common, common_num, current_common_num = get_recommendation_for_a_playlist(
        user_id, token, limit, name, description, playlist_id=playlist_id, songs_id = songs_id, create_pl=False, public=public, **kwargs)

    songs = [x if x not in common_songs else None for x in song_recom]
    songs_id = [x for x in songs if x is not None]
    market = kwargs.get("market", 'US')
    ## place common songs
    if common_num - current_common_num > 0:
        com, place = place_common(user_id, token, market = market, 
                                  common_songs = list(filter(lambda x: x not in used_common, common_songs)), 
                                  num = common_num - current_common_num, songs_id = songs_id, 
                                  pars_weight = pars_weight)
        ## replace recommendations with common songs
        count = 0
        for i in range(len(songs)):
            if songs[i] is not None:
                for j in range(len(place)):
                    if count == place[j]:
                        songs[i] = com[j]
                count +=1
            else:
                songs[i] = song_recom[i]
    else:
        songs = song_recom
    
    if create_pl:
        create_plst(user_id, token, uris=songs, name=name, description=description, public=public)

    return songs

### Place common songs first, then recommend

In [13]:
def PlaceRecommend(user_id, token, limit, name='',description='', playlist_id=None, songs_id=None, create_pl=False, public=False, **kwargs):
    market = kwargs.get("market", 'US')
    kwargs["market"] = market

    common_top = kwargs.get("common_top", 0.25) ## default: songs that have a frequency within the top 25% are regarded as common songs 
    start_plst = kwargs.get("start_plst", 0)
    end_plst = kwargs.get("end_plst", 20)
    common_songs = ['spotify:track:'+ x for x in common(common_top, user_id, token, market = market, start_plst = start_plst, end_plst = end_plst)]
    
    pars_weight = kwargs.get("pars_weight", {'energy': 0.4, 'duration_ms': 0.4, 'danceability': 0.2})
    kwargs.pop('pars_weight', None)
    
    if playlist_id is not None:
        ids, durations = get_ID(user_id, token, playlist_id, market)
        ids = ['spotify:track:'+ x for x in ids]
    elif songs_id is not None:
        ids = songs_id
    else:
        NameError('Please input either playlist_id or songs_id.')
    
    common_num = round(kwargs.get("common_prop", 0) * len(ids)) ## default: don't use common song to substitute recommendations
    kwargs.pop('common_prop', None)
    
    def meet_middle_for_bucket(user_id, token, limit, songs_id, **kwargs):
        f_pl, _ = get_recommendation_for_a_playlist(user_id, token, limit=limit, name='', description='', songs_id=songs_id, **kwargs)
        b_pl, _ = get_recommendation_for_a_playlist(user_id, token, limit=limit, name='', description='', songs_id=songs_id[::-1], **kwargs)
        b_pl = b_pl[::-1]
        if f_pl == b_pl or len(f_pl) == 1:
            return([x[14:] for x in f_pl])
        else:
            f_pl = [x[14:] for x in f_pl]
            b_pl = [x[14:] for x in b_pl]
            middle = len(f_pl) // 2
            energys = [get_song(user_id, token, x, market)['energy'][0]  for x in songs_id]
            trends = [1 if energys[i+1] > energys[i] else -1 for i in range(len(energys)-1)]
            f_energy = get_song(user_id, token, f_pl[middle-1], market)['energy'][0] 
            b_energy = get_song(user_id, token, b_pl[middle], market)['energy'][0]
            trend = trends[middle-1]
            if f_pl[middle] == b_pl[middle] or trend * (b_energy - f_energy) > 0:
                return(f_pl[:middle] + b_pl[middle:])
            else:
                offset = 1
                finished = None
                while trend * (b_energy - f_energy) <= 0 and middle+1+offset < len(b_pl)-1:
                    f_energy = get_song(user_id, token, f_pl[middle-1+offset], market)['energy'][0] 
                    b_energy = get_song(user_id, token, b_pl[middle+1+offset], market)['energy'][0]
                    trend = trends[middle-1+offset]
                    if trend * (b_energy - f_energy) > 0:
                        return(f_pl[:(middle+1)] + b_pl[(middle+1):])
                    else:
                        offset += 1
            if finished is None:
                offset = -1
                while trend * (b_energy - f_energy) <= 0 and middle-1+offset > 0:
                    f_energy = get_song(user_id, token, f_pl[middle-1+offset], market)['energy'][0] 
                    b_energy = get_song(user_id, token, b_pl[middle+1+offset], market)['energy'][0]
                    trend = trends[middle-1+offset]
                    if trend * (b_energy - f_energy) > 0:
                        return(f_pl[:(middle+1)] + b_pl[(middle+1):])
                    else:
                        offset += -1
            if finished is None:
                ### if after searching from the middle to the end and the beginning, no points between 
                ### these two playlists can be combined, we just use the original one that goes from the beginning
                return (f_pl)
         
    songs = [x if x not in common_songs else None for x in ids]
    song_id = [x for x in songs if x is not None]
    com, place = place_common(user_id, token, market = market, songs_id = song_id, 
                              common_songs = common_songs, num = common_num, pars_weight = pars_weight)

    ## replace target songs with common songs
    count = 0
    for i in range(len(songs)):
        if songs[i] is not None:
            for j in range(len(place)):
                if count == place[j]:
                    songs[i] = com[j]
            count +=1
        else:
            songs[i] = ids[i]

    sorted_place = sorted(set(place + [len(songs) - 1, 0]))
    buckets = [[songs[i] for i in range(sorted_place[j], sorted_place[j+1])] 
               if j < len(sorted_place)-2 else [songs[i] for i in range(sorted_place[j], sorted_place[j+1]+1)] 
               for j in range(len(sorted_place)-1)]
    buckets = [[x[14:] for x in lst]for lst in buckets]

    ## generate recommendations using meet middle method
    recommendations_pl = []
    for song_id in buckets:
        if len(song_id) > 2:
            pl = meet_middle_for_bucket(user_id, token, limit, song_id[1:-1], **kwargs)
            pl = [song_id[0]] + pl + [song_id[-1]]
        else:
            pl = song_id
        recommendations_pl.append(pl)

    ## flattern
    recommendations_pl = [num for elem in recommendations_pl for num in elem]
    ## add something so that Spotify API know what it is 
    recommendations_pl = ['spotify:track:'+ str(x) for x in recommendations_pl]
    
    if create_pl:
        create_plst(user_id, token, uris=recommendations_pl, name=name, description=description, public=public)
    
    return recommendations_pl

## Parameters

The number of possible parameters is too large to list all of them. I hope the explanation below can provide a basic idea for the parameters for you. 

#### 1. For function `get_recommendation_for_a_playlist`: 
- user_id
- token
- limit: the target size of the list of recommendations
- name: name for the new playlist
- description: description for the new playlist
- playlist_id: the id for the target playlist
- songs_id: a list of target song ids
- create_pl: if True, create a playlist with the final recommendations
- public: if True, the playlist to be created is a public playlist
- quiet: if False, print information during the process
- **kwargs**: There are many other optional parameters can be assigned. Assign it as `par_name = par_value`, such as `market = 'US'`. 
     - market: the country of the song market you want to explore
     - common_top: songs with a (weighted) frequency within top "common_top" will be regarded as common songs
     - start_plst: when getting common songs, start from which playlist. 0 means starting from the latest playlist
     - end_plst: when getting common songs, end at which playlist. 1 means ending from the second latest playlist
     - common_prop: the proportion of common songs you want to have. **This should be 0 or not provided in the pure method.**
     - percent_args: when getting recommendations using Spotify API, we need to submit parameters, including target value, minimum value and maximum value. **This should be a dictionary.** For example, `percent_args = {'time': 15, 'energy': 0.1}`.
          - 'time': in the above example, we will consider a duration interval with length 30s.
          - other parameters, including 'energy', 'instrumentalness', 'tempo', 'valence', 'speechiness', 'acousticness', 'liveness', 'danceability', 'key', 'loudness', 'mode', 'time_signature': percent range. In the above example, we will set `target_energy` as the target song's energy, `min_energy=(1-0.1)*target_energy` and `max_energy=(1+0.1)*target_energy`. 
          
Note: if both `playlist_id` and `songs_id` are given, we will use `playlist_id` and ignore `songs_id`. If neither is given, throw an error message. 

#### 2. For function `meet_middle`: 
- **bucket_lengths**: the number of minutes in each bucket, for example, `bucket_lengths=[10, 15]` means to create two buckets, one has a duration about 10 minutes and the other has a duration about 15 minutes.

Note: other parameters as the same as `get_recommendation_for_a_playlist`. 

#### 3. For function `RecommendPlace`: 
- **pars_weight**: the weight for each parameter when compare difference between songs. For example, `pars_weight={'energy': 0.4, 'duration_ms': 0.4, 'danceability': 0.2}` means that when calculating the difference between two songs, \begin{equation} diff = 0.4*(energy1-energy2)^2+0.4*(duration\_ms1-duration\_ms2)^2+0.2*(danceability1-danceability2)^2 \end{equation}

Note: other parameters as the same as `get_recommendation_for_a_playlist`. 

#### 4. For function `PlaceRecommend`: 
`common_top`, `start_plst`, `end_plst`, `common_prop` and `pars_weight` are required, instead of optional parameters.

Note: other parameters as the same as `get_recommendation_for_a_playlist`. 

## Test

Note: token should have scope at least "playlist-modify-private", you can find get one at the botton GET TOKEN from [here](https://developer.spotify.com/console/get-recommendations/?limit=10&market=ES&seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=classical%2Ccountry&seed_tracks=0c6xIDDpzE81m2q797ordA&min_acousticness=&max_acousticness=&target_acousticness=&min_danceability=&max_danceability=&target_danceability=&min_duration_ms=&max_duration_ms=&target_duration_ms=&min_energy=&max_energy=&target_energy=&min_instrumentalness=&max_instrumentalness=&target_instrumentalness=&min_key=&max_key=&target_key=&min_liveness=&max_liveness=&target_liveness=&min_loudness=&max_loudness=&target_loudness=&min_mode=&max_mode=&target_mode=&min_popularity=&max_popularity=&target_popularity=&min_speechiness=&max_speechiness=&target_speechiness=&min_tempo=&max_tempo=&target_tempo=&min_time_signature=&max_time_signature=&target_time_signature=&min_valence=&max_valence=&target_valence=).

User id can be obtained from [your Spotify profile](https://www.spotify.com/us/account/overview/?utm_source=spotify&utm_medium=menu&utm_campaign=your_account).

In [ ]:
# settings
token = "BQDdbJ0tx0Nz0O7fboxfugCaYyZgPVjoGbRzbqN6abdNiAySIExSL06jeODo0O7FtNree79PlRTXRZHI1recFIxqed6vgUXV3ySwZCdq2dlvxAtVmRU-Bl5ZpW7S7-4n-_rAMlm_0bn3dm0AZilqIv6-1S-RgG3VvZP6PUc4Um2pT4Fw8w2qLI7S8bJGoKAeJE4sYN61LrBCRw"
user_id = "pbwppse1hilahmk43ls424ao4"

In [ ]:
## for test use only
target_songs = ['spotify:track:7bzks4LGpQUuPKBzJ6iQ7y',
 'spotify:track:7nBR4Tt431p1MTgv3lVsmX',
 'spotify:track:4VAdq9M22v99JHIXNv8TZ6',
 'spotify:track:5prMnA9apLCmFBjL9sbpUR',
 'spotify:track:6mybKC52hIM1WYfp73CaOl',
 'spotify:track:0dbqPssLj7KzNTH81tcsrZ',
 'spotify:track:7caJcFZTtLzy0ZSol1AXKH',
 'spotify:track:7JPjG8J0sXCIh9KkWBy4vw',
 'spotify:track:09RXXnMSVrES6xju7IXrsX',
 'spotify:track:6IBCA5bu9eIBOsaoO0MqJj',
 'spotify:track:4MvbRbrOEsJgdYRGNGBjTE']

#### Test pure method

In [ ]:
name = 'pure_pl'
description = ''
limit = 10  ## for each song in current playlist, 10 recommendations are generated
market = 'US'

playlist_id = '4N2UHn9HpFc3n93s1gduIM'
percent_args = {'time': 15, 'energy': 0.1} #'liveness': 0.05

In [ ]:
song_recom, candidate = get_recommendation_for_a_playlist(user_id, token, limit, name, description, 
                                                          playlist_id = playlist_id, create_pl=True, market=market, 
                                                          percent_args = percent_args, quiet = True)

In [ ]:
song_recom

In [ ]:
name = 'pure_songs'
description = ''
song_recom, candidate = get_recommendation_for_a_playlist(user_id, token, limit, name, description, 
                                                          songs_id = target_songs, create_pl=True, market='US', 
                                                          percent_args = percent_args, quiet = True)

In [ ]:
song_recom

#### Test Meet Middle

In [ ]:
name = 'meetmiddle_pl'
description = ''
r_meetmiddle = meet_middle(user_id, token, bucket_lengths = [15, 20, 2], limit=limit, name=name, description=description, 
                           playlist_id = playlist_id, market = 'US', percent_args = percent_args, create_pl=True)

In [ ]:
r_meetmiddle

In [ ]:
name = 'meetmiddle_songs'
description = ''
r_meetmiddle = meet_middle(user_id, token, bucket_lengths = [5, 15, 25, 13], limit=limit, name=name, description=description, 
                           songs_id = target_songs, market = 'US', percent_args = percent_args, create_pl=True)

In [ ]:
r_meetmiddle

#### Test "Recommend first, then place common songs"

In [ ]:
## all possible parameters
## ['energy', 'duration_ms', 'instrumentalness', 'danceability', 'key', 'loudness', 
## 'mode', 'speechiness', 'acousticness', 'liveness', 'valence', 'tempo', 'time_signature']

pars_weight = {'energy': 0.4, 'duration_ms': 0.4, 'danceability': 0.2}
name = 'recomplace_pl'
description = ''
r_recommendplace = RecommendPlace(user_id, token, limit, name, description, 
                                  playlist_id=playlist_id, 
                                  market='US', percent_args = percent_args, pars_weight = pars_weight, quiet = True, 
                                  common_top = 0.6, start_plst = 2, end_plst = 10, common_prop = 0.2, create_pl=True)

In [ ]:
r_recommendplace

In [ ]:
name = 'recomplace_songs'
description = ''
r_recommendplace = RecommendPlace(user_id, token, limit, name, description, market='US', percent_args = percent_args, 
                                  songs_id = target_songs, common_top = 0.6, start_plst = 2, end_plst = 10, common_prop = 0.2, 
                                  pars_weight = pars_weight, quiet = True, create_pl=True)

In [ ]:
r_recommendplace

#### Test "Place common songs first, then recommend"

In [ ]:
pars_weight = {'energy': 0.4, 'duration_ms': 0.4, 'danceability': 0.2}
name = 'placerecom_pl'
description = ''
r_placerecommend = PlaceRecommend(user_id, token, limit, name, description, playlist_id = playlist_id, 
               market = 'US', pars_weight=pars_weight, percent_args = percent_args, 
               common_top = 0.6, start_plst = 2, end_plst = 10, common_prop = 0.2, quiet = True, create_pl=True)

In [ ]:
r_placerecommend

In [ ]:
name = 'placerecom_songs'
description = ''
r_placerecommend = PlaceRecommend(user_id, token, limit, name, description, songs_id = target_songs, 
               market = 'US', pars_weight=pars_weight, percent_args = percent_args, 
               common_top = 0.6, start_plst = 2, end_plst = 10, common_prop = 0.2, quiet = True, create_pl=True)

In [ ]:
r_placerecommend

## ======================  Appendix =========================================
#### Test for song frequency

In [ ]:
start_plst = 0 ## 0 means start from the most recent one
end_plst = 10 ## 10 means end at the 10 recent one
## in this case, 11 playlists are used 

#### Use create time for playlist

In [ ]:
df = get_song_freq(user_id, token, market = 'US', start_plst = 0, end_plst = 20, use_time = False)
df.head(10)

#### Don't use create time for playlist

In [ ]:
df1 = get_song_freq(user_id, token, market = 'US', start_plst = 0, end_plst = 20, use_time = True, decay_rate = 15)
df1.head(10)